# BIM/BM25 기반 한국어 검색 시스템 구현 및 모델 비교 분석

**학번:** 2170045  
**학과:** 컴퓨터공학과  
**이름:** 서자영

---

## 1. 프로젝트 개요

본 프로젝트는 정보 검색(Information Retrieval)의 대표적인 확률 모델인 BIM(Binary Independence Model)과 BM25를 직접 구현하고, 한국어 데이터셋(KomuRetrieval)을 활용하여 두 모델의 성능 차이를 체계적으로 비교 분석한다. 검색 성능에 영향을 미치는 주요 변수들을 세 가지 회귀 분석 방법론—이진 로지스틱 회귀(Binary Logistic Regression), 다항 로지스틱 회귀(Multinomial Logistic Regression), 선형 다변량 회귀(Linear Multinomial Regression)—을 통해 정량적으로 분석하고, 각 변수의 통계적 유의성을 검증하였다.

### 1.1 연구 목표

1. **모델 비교**: BIM과 BM25의 한국어 검색 성능 비교 (MAP, P@10, R@10)
2. **세부 그룹 분석**: 문서 길이(Length), 토픽(Topic), 쿼리 복잡도(Complexity)별 성능 차이 검증
3. **통계적 검증**: 회귀 모델 구축을 통한 변수별 영향력 정량화 및 편향성 분석
4. **하이퍼파라미터 최적화**: BM25의 k1, b 파라미터 튜닝을 통한 한국어 데이터 최적화

### 1.2 개발 환경

- **언어**: Python 3.11.9
- **IDE**: IntelliJ IDEA
- **주요 라이브러리**:
  - `kiwipiepy`: 한국어 형태소 분석
  - `gensim`: LDA 토픽 모델링
  - `statsmodels`: 로지스틱 회귀 및 OLS 회귀 분석
  - `scikit-learn`: 데이터 분할 및 평가 지표
  - `SQLite`: 역색인 저장 및 검색

## 2. 데이터셋 및 실험 환경

### 2.1 데이터셋

- **Corpus**: KomuRetrieval 전체 50,222개 문서 중 5,000개 샘플링
- **Queries**: 1,454개 질의 집합
- **Qrels**: Query-Document 관련성 레이블

### 2.2 스마트 샘플링 (Smart Sampling)

단순 무작위 추출 시 정답 문서가 누락되는 문제를 방지하기 위해, qrels에 포함된 문서를 우선 포함하는 전략을 사용하였다.

```
전체 문서: 50,222개
정답 포함 문서: 6,194개
최종 샘플: 5,000개 (정답 문서 우선 + 무작위)
```

### 2.3 데이터 전처리 및 불용어 처리

#### 2.3.1 IDF 기반 불용어 선정

5,000개 문서 전체에서 Document Frequency(DF)가 높고 IDF < 1.5인 단어를 불용어로 지정하였다.

**해석**: "나오", "보이" 등은 5,000개 문서 중 3,000개 이상에 등장하여 IDF가 0.26~0.47로 매우 낮아 검색 변별력이 없다.

#### 2.3.2 패딩 처리

불용어를 삭제하지 않고 글자 수만큼 'O'로 치환하여 문서 길이를 보존하였다.

```python
"나오다" (3글자) → "OOO" (3글자, 길이 유지)
```

### 2.4 문서 특성 추출

**주요 발견:**
- 문서 길이 편차가 매우 큼 (최소 0 ~ 최대 102,419자)
- 평균 문장 길이: 55.7 음절 (중앙값 47)
- 이러한 길이 분포가 BIM/BM25 성능 차이의 주요 원인

### 2.5 LDA 토픽 모델링

## 3. 시스템 구현

### 3.1 역색인 구축

SQLite를 사용하여 대용량 데이터 처리와 메모리 효율성을 동시에 확보하였다.

### 3.2 BM25 하이퍼파라미터 튜닝

BM25의 k1(TF saturation)과 b(length normalization) 파라미터를 Grid Search를 통해 MAP 기준으로 최적화하였다.

In [ ]:
import os
import pandas as pd
import numpy as np
from IPython.display import Image, display
import warnings
warnings.filterwarnings('ignore')

BASE_DIR = os.path.join('..')
DATA_DIR = os.path.join(BASE_DIR, 'data_final')

tuning_coarse_path = os.path.join(DATA_DIR, 'tuning_coarse_v2.csv')
if os.path.exists(tuning_coarse_path):
    df_tuning_coarse = pd.read_csv(tuning_coarse_path)
    print("\n[1차 탐색 상세 결과]")
    display(df_tuning_coarse.head(10))

tuning_fine_path = os.path.join(DATA_DIR, 'tuning_fine_v2.csv')
if os.path.exists(tuning_fine_path):
    df_tuning_fine = pd.read_csv(tuning_fine_path)
    print("\n[2차 탐색 상세 결과]")
    display(df_tuning_fine)

**해석:**
- **b=0.99**: 문서 길이 정규화를 최대로 적용
- **k1=3.25**: 단어 빈도(TF) 증가에 따른 점수 상승 폭을 완만하게 설정

---

## 4. 통계적 분석 1: 전역 회귀 분석 (Global Regression Analysis)

### 4.1 변수 정의 및 다중공선성 진단

검색 성공 여부 또는 검색 점수를 종속변수로, 다음 3개 독립변수를 선정하였다:

- **x1 (a1)**: `doc_length_scaled` - 문서 길이 / 1000
- **x2 (a2)**: `query_length_scaled` - 쿼리 길이 (형태소 개수)
- **x3 (a3)**: `dominant_topic` - 문서의 주도 토픽 (0~9)

In [2]:
vif_full_path = os.path.join(DATA_DIR, 'vif_full_v2.csv')
if os.path.exists(vif_full_path):
    df_vif_full = pd.read_csv(vif_full_path)
    print("[다중공선성 진단 - 전체 변수]")
    display(df_vif_full)
    print("\nVIF > 10: 다중공선성 심각")

vif_final_path = os.path.join(DATA_DIR, 'vif_final_v2.csv')
if os.path.exists(vif_final_path):
    df_vif_final = pd.read_csv(vif_final_path)
    print("\n[최종 변수 VIF]")
    display(df_vif_final)
    print("\n모든 변수가 VIF < 3으로 다중공선성 문제 없음")


[1차 탐색 상세 결과]


,k1,b,MAP
0,2.0,0.60,0.604511
1,2.0,0.70,0.611607
2,2.0,0.80,0.619449
3,2.0,0.90,0.625556
4,2.0,0.99,0.627744
5,2.5,0.60,0.606648
6,2.5,0.70,0.614600
7,2.5,0.80,0.620384
8,2.5,0.90,0.626270
9,2.5,0.99,0.629283



[2차 탐색 상세 결과]


,k1,b,MAP
0,2.50,0.89,0.626547
1,2.50,0.94,0.628715
2,2.50,0.99,0.629283
3,2.75,0.89,0.627126
4,2.75,0.94,0.630104
5,2.75,0.99,0.629780
6,3.00,0.89,0.627705
7,3.00,0.94,0.630299
8,3.00,0.99,0.630534
9,3.25,0.89,0.628346


### 4.2 이진 로지스틱 회귀 (Binary Logistic Regression)

관련성(relevance: 0 또는 1)을 종속변수로 하는 로지스틱 회귀를 수행하여 Odds Ratio를 계산하였다.

#### 회귀 모델

```
logit(P(relevance=1)) = β0 + β1*doc_length + β2*query_length + Σβ3_i*topic_i
```

In [ ]:
odds_path = os.path.join(DATA_DIR, 'odds_ratio_v2.csv')
if os.path.exists(odds_path):
    df_odds = pd.read_csv(odds_path)
    
    print("[주요 변수 Odds Ratio]")
    main_vars = df_odds[df_odds['Feature'].isin(['doc_length_scaled', 'query_length_scaled'])]
    display(main_vars[['Model', 'Feature', 'Odds_Ratio', 'P_value', 'Lower_CI', 'Upper_CI']])
    
    print("\n[토픽별 Odds Ratio (유의미한 토픽만)]")
    topic_vars = df_odds[df_odds['Feature'].str.startswith('topic_')]
    significant_topics = topic_vars[topic_vars['P_value'] < 0.05]
    display(significant_topics[['Model', 'Feature', 'Odds_Ratio', 'P_value', 'Lower_CI', 'Upper_CI']])

In [ ]:
odds_comparison_img = os.path.join(DATA_DIR, 'odds_ratio_comparison_v2.png')
if os.path.exists(odds_comparison_img):
    print("\n[Odds Ratio 시각화]")
    display(Image(odds_comparison_img))

#### Odds Ratio 해석

**a1 (문서 길이):**
- **BIM**: OR=0.89 → 문서 1,000자 증가 시 성공 확률 11% 감소 (짧은 문서 편향)
- **BM25**: OR=1.01 → 문서 1,000자 증가 시 성공 확률 0.86% 증가 (거의 중립적)

**a2 (쿼리 길이):**
- **BIM**: OR=1.07 → 형태소 1개 증가 시 성공 확률 6.9% 증가
- **BM25**: OR=1.03 → 형태소 1개 증가 시 성공 확률 3.3% 증가

#### 검색 성능 비교

In [ ]:
summary_path = os.path.join(DATA_DIR, 'summary_metrics_v2.csv')
if os.path.exists(summary_path):
    df_summary = pd.read_csv(summary_path)
    print("[모델별 성능 요약]")
    display(df_summary)

In [ ]:
perf_comparison_img = os.path.join(DATA_DIR, 'performance_comparison_v2.png')
if os.path.exists(perf_comparison_img):
    print("\n[성능 비교 시각화]")
    display(Image(perf_comparison_img))

**종합 해석:**
- 검색 성능: BM25가 모든 지표에서 압도적 우위 (MAP +51.5%)
- Pseudo R²: BIM 높음(0.23) = 길이만으로 예측 가능 = 편향
- Best AUC: BIM 높음(0.83) = 길이 편향으로 성공 예측 가능 = 불공정

### 4.3 가중치 튜닝 (Manual Linear Scoring)

In [ ]:
weight_tuning_path = os.path.join(DATA_DIR, 'weight_tuning_v2.csv')
if os.path.exists(weight_tuning_path):
    df_tuning = pd.read_csv(weight_tuning_path)
    
    print("[BIM 최적 가중치]")
    bim_top = df_tuning[df_tuning['model']=='BIM'].sort_values('AUC', ascending=False).head(5)
    display(bim_top)
    
    print("\n[BM25 최적 가중치]")
    bm25_top = df_tuning[df_tuning['model']=='BM25_Best'].sort_values('AUC', ascending=False).head(5)
    display(bm25_top)

In [ ]:
weight_heatmap_img = os.path.join(DATA_DIR, 'weight_heatmap_v2.png')
if os.path.exists(weight_heatmap_img):
    print("\n[가중치 히트맵]")
    display(Image(weight_heatmap_img))

**핵심 해석:**

**BIM:**
- w1=-2.0 (문서 길이에 음수 가중치) → 짧은 문서 극단적 편향
- w3=0.0 (토픽 무시)
- AUC=0.83 → 높은 AUC = 편향이 심하다는 증거

**BM25:**
- w1=1.0 (문서 길이 긍정, 중립적)
- w2=3.0 (쿼리 길이 강조)
- w3=0.0 (토픽 무시)
- AUC=0.58 → 낮은 AUC = 길이 편향 없음 = 공정함

### 4.4 선형 다변량 회귀 (Linear Multinomial Regression)

검색 점수(search_score) 자체를 종속변수로 하는 다중 선형 회귀를 수행하였다. 토픽은 더미 변수(dummy variables)로 변환하여 각 토픽의 개별 효과를 측정하였다.

#### 회귀 모델

```
y = a1*x1 + a2*x2 + a3_1*topic1 + a3_2*topic2 + ... + a3_9*topic9 + b
```

여기서 topic 0을 기준(reference)으로 설정하고, topic 1~9의 상대적 효과를 측정한다.

In [ ]:
linear_results_path = os.path.join(DATA_DIR, 'linear_regression_results_v5.csv')
if os.path.exists(linear_results_path):
    df_linear = pd.read_csv(linear_results_path)
    print("[Linear Regression 모델 성능]")
    display(df_linear)

In [ ]:
linear_stats_path = os.path.join(DATA_DIR, 'linear_regression_stats_v5.csv')
if os.path.exists(linear_stats_path):
    df_linear_stats = pd.read_csv(linear_stats_path)
    print("\n[회귀 계수 및 통계적 유의성]")
    display(df_linear_stats)

**회귀 계수 해석:**

**a1 (문서 길이):**
- **BIM**: -0.0258 *** → 문서 1,000자 증가 시 점수 0.026 감소 (짧은 문서 선호)
- **BM25**: +0.0590 ** → 문서 1,000자 증가 시 점수 0.059 증가 (긴 문서 약간 선호)

**a2 (쿼리 길이):**
- **BIM**: +0.6889 *** → 쿼리 1 증가 시 점수 0.69 증가
- **BM25**: +1.3221 *** → 쿼리 1 증가 시 점수 1.32 증가 (더 민감)

**a3 (토픽 효과):**

In [ ]:
linear_topic_path = os.path.join(DATA_DIR, 'linear_regression_topic_coefs_v5.csv')
if os.path.exists(linear_topic_path):
    df_topic_coefs = pd.read_csv(linear_topic_path)
    
    print("[BIM 토픽 계수 (유의미한 토픽만)]")
    bim_topics = df_topic_coefs[(df_topic_coefs['Model'] == 'BIM') & (df_topic_coefs['P_value'] < 0.05)]
    bim_topics_sorted = bim_topics.sort_values('Coefficient', ascending=False)
    display(bim_topics_sorted[['Topic', 'Coefficient', 'P_value', 'CI_Lower', 'CI_Upper']])
    
    print("\n[BM25 토픽 계수 (유의미한 토픽만)]")
    bm25_topics = df_topic_coefs[(df_topic_coefs['Model'] == 'BM25_Best') & (df_topic_coefs['P_value'] < 0.05)]
    bm25_topics_sorted = bm25_topics.sort_values('Coefficient', ascending=False)
    display(bm25_topics_sorted[['Topic', 'Coefficient', 'P_value', 'CI_Lower', 'CI_Upper']])

**핵심 발견:**

1. **토픽의 유의미한 영향**: Binary Logistic에서는 w3=0.0이었지만, Linear Regression에서는 대부분의 토픽이 통계적으로 유의미함
   
2. **Topic 7 (마인크래프트)의 강력한 효과**:
   - BIM: +1.66점
   - BM25: +6.90점 (약 4배!)
   
3. **Binary vs Linear의 차이**:
   - Binary Logistic (w3=0): 관련성(0/1) 판단에는 토픽 무용
   - Linear (유의미): 검색 점수 자체에는 토픽 영향 있음

#### 시각화 분석

In [ ]:
linear_viz_path = os.path.join(DATA_DIR, 'linear_regression_v5.png')
if os.path.exists(linear_viz_path):
    print("[Linear Regression 종합 시각화]")
    display(Image(linear_viz_path))
else:
    print("시각화 파일을 찾을 수 없습니다.")

**시각화 해석:**

- **주요 변수 계수 비교 (좌상단)**: BIM의 a1(doc)이 음수, BM25는 양수
- **모델 설명력 (중상단)**: BIM의 Test R²가 BM25보다 약 2배 높지만, 이는 편향의 증거
- **T-통계량 비교 (우상단)**: 두 모델 모두 a2(query_length)의 t값이 매우 높음
- **토픽별 계수 (중하단)**: 초록색은 유의미한 토픽 (p < 0.05)
- **Actual vs Predicted (하단)**: BIM의 높은 설명력은 편향된 예측에서 기인

---

## 5. 통계적 분석 2: 세부 그룹별 심층 분석 (Sub-group Analysis)

### 5.1 문서 길이별 분석 (Length Analysis)

In [ ]:
length_perf_path = os.path.join(DATA_DIR, 'length_performance_v3.csv')
if os.path.exists(length_perf_path):
    df_length = pd.read_csv(length_perf_path)
    print("[문서 길이별 성능 요약 (MAP)]")
    display(df_length.pivot_table(index='category', columns='model', values='MAP'))

In [ ]:
length_map_img = os.path.join(DATA_DIR, 'length_map_trend_v3.png')
if os.path.exists(length_map_img):
    print("\n[길이별 MAP 변화]")
    display(Image(length_map_img))

**핵심 발견:**

1. **Long 문서**: BIM 성능은 0.593으로 급락, BM25는 0.824로 최고 성능
   - 성능 차이: +0.231 (39% 개선)
   
2. **BIM의 성능 저하 패턴**: Short → Medium → Long으로 갈수록 하락
   
3. **BM25의 성능 향상 패턴**: Short → Medium → Long으로 갈수록 상승

### 5.2 토픽별 분석 (Topic Analysis)

In [ ]:
topic_perf_path = os.path.join(DATA_DIR, 'topic_performance_v3.csv')
if os.path.exists(topic_perf_path):
    df_topic = pd.read_csv(topic_perf_path)
    print("[토픽별 성능 요약 (MAP)]")
    display(df_topic.pivot_table(index='topic', columns='model', values='MAP'))

In [ ]:
topic_map_img = os.path.join(DATA_DIR, 'topic_map_trend_v3.png')
if os.path.exists(topic_map_img):
    print("\n[토픽별 MAP 변화]")
    display(Image(topic_map_img))

**핵심 발견:**

1. **Topic 2 (스포츠)의 낮은 성능**: 두 모델 모두 가장 낮음 (BIM: 0.331, BM25: 0.636)
2. **Topic 6, 7, 9의 높은 성능**: BM25 MAP 0.8 이상
3. **BM25의 일관된 우위**: 모든 토픽에서 BM25 > BIM

### 5.3 쿼리 복잡도별 분석 (Complexity Analysis)

In [ ]:
complexity_perf_path = os.path.join(DATA_DIR, 'complexity_performance_v3.csv')
if os.path.exists(complexity_perf_path):
    df_complexity = pd.read_csv(complexity_perf_path)
    print("[쿼리 복잡도별 성능 요약 (MAP)]")
    display(df_complexity.pivot_table(index='complexity', columns='model', values='MAP'))

In [ ]:
complexity_map_img = os.path.join(DATA_DIR, 'complexity_map_trend_v3.png')
if os.path.exists(complexity_map_img):
    print("\n[복잡도별 MAP 변화]")
    display(Image(complexity_map_img))

**핵심 발견:**

1. **Complex 쿼리**: BIM 0.514 vs BM25 0.764 (+0.250, 49% 개선)
2. **BIM의 구조적 한계**: Binary 매칭으로 긴 쿼리 처리 어려움
3. **BM25의 유연성**: 가중치 합산 방식으로 긴 쿼리 효과적 처리

### 5.4 종합 시각화

In [ ]:
additional_img = os.path.join(DATA_DIR, 'additional_analysis_v3.png')
if os.path.exists(additional_img):
    print("[세부 분석 종합 시각화]")
    display(Image(additional_img))

---

## 6. 도메인/토픽 변수 심층 분석

Linear Regression에서는 토픽이 유의미한 영향을 보였으나, Binary Logistic과 가중치 튜닝에서는 w3=0.0으로 수렴하였다. 이 모순을 해결하기 위해 실제 검색 케이스 10개를 정성적으로 분석하였다.

### 6.1 분석 방법

1. 유효한 쿼리 중 무작위 10개 추출
2. BM25 (k1=3.25, b=0.99) 검색 결과 분석
3. 정답 문서와 오답 문서의 토픽 분포 비교

### 6.2 패턴 분석

#### 패턴 A: 토픽 변별력 부재 (8개 케이스)

정답 문서들이 동일한 토픽에 속하지만, 오답 문서들도 같은 토픽. 토픽 가중치를 높이면 주제만 같고 내용은 다른 오답이 상위로 난입.

#### 패턴 B: 토픽 불일치 (2개 케이스)

정답 문서들이 서로 다른 토픽으로 분류됨. 특정 토픽에 가중치를 주면 다른 토픽의 정답 문서들이 순위 밖으로 밀려남.

### 6.3 결론

**w3 (토픽 가중치) = 0.0인 이유:**
1. 변별력 부재: 토픽은 정답과 오답을 구별하지 못함
2. 키워드의 압도적 우위: BM25만으로 정답을 1~3위에 정확히 배치
3. 방해 요소: 토픽 정보는 오히려 검색 품질을 저하시킬 가능성

**Linear에서 토픽이 유의미했던 이유:**
- Linear는 검색 점수 자체를 예측
- Topic 7은 특정 어휘가 많아 검색 점수가 높아지는 경향
- 하지만 관련성(0/1) 판단에는 도움 안 됨

---

## 7. 결론

### 7.1 연구 결과 요약

#### 검색 성능 최종 비교

In [ ]:
print("[검색 성능 최종 비교]")
perf_data = {
    '평가 지표': ['MAP', 'P@10', 'R@10', 'MAP 표준편차', 'P@10 표준편차', 'R@10 표준편차'],
    'BIM': [0.416, 0.193, 0.520, 0.343, 0.189, 0.376],
    'BM25': [0.631, 0.270, 0.681, 0.325, 0.224, 0.314],
    '개선율': ['+51.5%', '+40.0%', '+31.0%', '-5.3%', '+18.5%', '-16.5%']
}
df_perf = pd.DataFrame(perf_data)
display(df_perf)

#### 회귀 분석 결과 종합

**1. Binary Logistic Regression:**
- BIM: Pseudo R²=0.234, Doc Length OR=0.89 (짧은 문서 편향)
- BM25: Pseudo R²=0.016, Doc Length OR=1.01 (중립)

**2. Multinomial Logistic Regression:**
- Long 문서: BIM MAP 0.593 vs BM25 MAP 0.824 (+39%)
- Complex 쿼리: BIM MAP 0.514 vs BM25 MAP 0.764 (+49%)

**3. Linear Multinomial Regression:**
- BIM: a1=-0.026 *** (짧은 문서 선호), Test R²=0.501
- BM25: a1=+0.059 ** (긴 문서 약간 선호), Test R²=0.262
- 토픽: 검색 점수에는 유의미, 관련성 판단에는 무용

### 7.2 최종 결론

BM25의 길이 정규화 메커니즘(b=0.99)이 한국어 비정형 텍스트 환경에서 검색 품질을 결정하는 핵심 요소임을 확인하였다.

**주요 발견:**
1. BM25는 BIM 대비 MAP 51.5% 개선
2. BIM의 짧은 문서 편향을 통계적으로 입증 (OR=0.89, a1=-0.026)
3. BM25의 공정성 확인 (OR=1.01, a1=+0.059)
4. 토픽은 검색 점수에 영향주나 관련성 판단에는 무용
5. Long 문서와 Complex 쿼리에서 BM25 압도적 우위

**실무적 시사점:**
- 문서 길이 편차가 큰 환경에서 b=0.99 권장
- 불용어 패딩 처리로 길이 정보 보존
- 토픽은 랭킹 피처보다 후처리/추천에 활용

---

## 부록: 데이터 파일 목록

In [ ]:
print("[생성된 데이터 파일]")
data_files = []

for file in os.listdir(DATA_DIR):
    if file.endswith('.csv') or file.endswith('.pkl') or file.endswith('.txt'):
        file_path = os.path.join(DATA_DIR, file)
        size = os.path.getsize(file_path)
        data_files.append({'파일명': file, '크기(bytes)': size})

df_files = pd.DataFrame(data_files).sort_values('파일명')
display(df_files)

print("\n[생성된 시각화 파일]")
viz_files = []

for file in os.listdir(DATA_DIR):
    if file.endswith('.png'):
        file_path = os.path.join(DATA_DIR, file)
        size = os.path.getsize(file_path)
        viz_files.append({'파일명': file, '크기(bytes)': size})

df_viz = pd.DataFrame(viz_files).sort_values('파일명')
display(df_viz)

---

**작성일:** 2025년 12월 13일  
**학번:** 2170045  
**이름:** 서자영